In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 1,
    length_function = len,
    is_separator_regex = False,
)

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
loader = PyPDFLoader('1-Maharashtra Rent Control Act-1999.pdf')
docs = loader.load_and_split()

In [8]:
texts = text_splitter.create_documents([each.page_content for each in docs])

In [9]:
len(texts)

293

In [10]:
# @title Region { display-mode: "form" }
REGION = "US"  # @param {type: "string"}

In [11]:
# @title Dataset and Table { display-mode: "form" }
DATASET = "vector_db"  # @param {type: "string"}
TABLE = "vector_tbl"  # @param {type: "string"}

In [12]:
# @title Project { display-mode: "form" }
PROJECT_ID = "image-genai"  # @param {type:"string"}

In [13]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

In [14]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)

Dataset(DatasetReference('image-genai', 'vector_db'))

In [15]:
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

store = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
    distance_strategy=DistanceStrategy.COSINE,
)

In [16]:
data = [t.page_content for t in texts]
metadatas = [{"len": len(t.page_content)} for t in texts]

ids = store.add_texts(data, metadatas=metadatas)

print(len(ids))

293


In [17]:
query = "Where is act being applied?"
query_vector = embedding.embed_query(query)
docs = store.similarity_search_by_vector(query_vector, k=3)
print(docs)

[Document(page_content='(b) this Act shall apply to any premises let for any or all purposes specified\nin sub-section ( 1) in the areas other than those specified in Schedule I and\nSchedule II.\n3. (1) This Act shall not apply—\n(a) to any premises belonging to the Government or a local authority or\napply as against the Government to any tenancy, licence or other like relationship\ncreated by a grant from or a licence given by the Government in respect of', metadata={'len': 432, '__id': 'fa37b863a77c4eb9bef5709be5d49db5'}), Document(page_content='46. Pending suits and proceedings in Courts.\n47. Bar of Jurisdiction.\n48. Section 18 to apply with modification.\n49. Competent Authority to be deemed to be public servant.\n50. All proceedings before Competent Authority to be judicial proceedings.\n51. Competent Authority to be deemed to be Civil Court for certain purposes.\n52. Protection of action taken under this Act.\n53. Certain offences to be cognizable.\n54. Offences by companies,

In [18]:
def augment_prompt(query: str):
    query_vector = embedding.embed_query(query)
    # get top 3 results from knowledge base
    results = store.similarity_search_by_vector(query_vector, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the question in detail.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [19]:
from vertexai.preview.generative_models import GenerativeModel
import re

def generate(query):
    prompt = augment_prompt(query)
    model = GenerativeModel("gemini-1.0-pro-vision-001")
    responses = model.generate_content(
        prompt,
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.7,
            "top_p": 1
        },
        stream=True,
    )

    res = []

    for response in responses:
        print(response.text, end="")
        

In [ ]:
generate("Where is act being applied?")

In [43]:
generate("What is this act about?")

This act is about the control of rent and eviction and other connected matters in the repealed Acts.

In [50]:
query = '''1) Period: That the Licensor hereby grants to the Licensee herein a revocable leave and license,
to occupy the Licensed Premises, described in Schedule I hereunder written without creating any
tenancy rights or any other rights, title and interest in favour of the Licensee for a period of 11
Months commencing from 10/04/2018 and ending on 09/03/2019'''

In [53]:
generate(query)

**Clause Meaning:**

If the person who is allowed to live in a property (the licensee) does not leave the property when their permission to live there (the license) expires, the landlord can ask a special authority (the Competent Authority) to order the licensee to leave. the landlord can ask a special authority (the Competent Authority) to help them get the property back.

In [54]:
query = '''2) License Fee & Deposit:That the Licensee shall pay to the Licensor License fee at the rate of
Rs. 21000(Twenty-One Thousand Only) per month towards the compensation and Rs. 50000(Fifty
Thousand Only) interest free refundable deposit, for the use of the said Licensed premises. The
amount of monthly compensation License fee shall be payable within first five days of the
concerned month of Leave and License.'''

In [55]:
generate(query)

**Clause:** The Licensee shall pay to the Licensor License fee at the rate of Rs. 21000(Twenty-One Thousand Only) per month towards the compensation and Rs. 50000(Fifty Thousand Only) interest free refundable deposit, for the use of the said Licensed premises. The amount of monthly compensation License fee shall be payable within first five days of the concerned month of Leave and License.

**Meaning:**

The person who is renting the property (Licensee) must pay the owner of the property (Licensor) a monthly fee of Rs. 21,000 as compensation for using the property. They must also pay a refundable deposit of Rs. 50,000. The monthly fee must be paid within the first five days of each month. days of each month that the Licensee is using the property.

In [56]:
query = '''4) Maintenance Charges: That the all outgoings including all rates, taxes, levies, assessment,
maintenance charges, non occupancy charges, etc. in respect of the said premises shall be paid
by the Licensor.'''

In [57]:
generate(query)

The clause means that the landlord is responsible for paying all the costs associated with the property, such as taxes, fees, and maintenance charges.

In [71]:
query = '''what are some actions I can take if this clause in breached? 6) Use: That the Licensed premises shall only be used by the Licensee for Residential
purpose.The Licensee shall maintain the said premises in its existing condition and damage, if any,
caused to the said premises, the same shall be repaired by the Licensee at its own cost subject to
normal wear and tear. The Licensee shall not do anything in the said premises which is or is likely
to cause a nuisance to the other occupants of the said building or to the prejudice in any manner to
the rights of Licensor in respect of said premises or shall not do any unlawful activities prohibited
by State or Central Government.'''

In [72]:
print(augment_prompt(query))

Using the contexts below, answer the question.

    Contexts:
    tenant or, any person residing with the tenant has been guilty of conduct which is a 
nuisance or annoyance to the adjoining or neighbouring occupier, or has been convicted 
of using the premises or allowing the premises to be used for immoral or illegal pur poses 
or that the tenant has in respect of the premises been convicted of an offence of 
contravention of any of the provisions of clause (a) of sub-section (1) of section 394 or of
Authority, on being satisfied that the period of licence has expired, shall pass an order  for 
eviction of a licensee. 
(2) Any licensee who does not deliver possession of the premises to the landlord on 
expiry of the period of licence and continues to be in possession of the licensed premises  
till he is dispossessed by the Competent Authority shall be liable to pay damages a t 
double the rate of the licence fee or charge of the premises fixed under the agree ment of 
licence.
Expla

In [74]:
generate(query)

The provided context does not contain any information about actions that can be taken if the clause is breached. "That the Licensed premises shall only be used by the Licensee for Residential purpose" is breached.

In [61]:
query = '''7) Alteration: That the Licensee shall not make or permit to do any alteration or addition to the
construction or arrangements (internal or external) to the Licensed premises without previous
consent in writing from the Licensor.
8) No Tenancy: That the Licensee shall not claim any tenancy right and shall not have any right to
transfer, assign, and sublet or grant any license or sub-license in respect of the Licensed Premises
or any part thereof and also shall not mortgage or raise any loan against the said premises.
LEAVE AND LICENSE AGREEMENT

Registered as Document No.5604/2018 at the Joint S.R. Haveli 14 on 20/04/2018 Page 2 of 6

9) Inspection: That, the Licensor shall on reasonable notice given by the Licensor to the Licensee
shall have a right of access either by himself / herself / themselves or through authorized
representative to enter, view and inspect the Licensed premises at reasonable intervals.
10) Cancellation: That, Subject to the condition of lock in period (if any), if the Licensee commits
default in regular and punctual payments of monthly compensation as herein before mentioned or
commit/s breach of any of the terms, covenants and conditions of this agreement or if any
legislation prohibiting the Leave and License is imposed, the Licensor shall be entitled to revoke
and / or cancel the License hereby granted, by giving notice in writing of one month and the
Licensee too will have the right to vacate the said premises by giving a notice in writing of one
month to the Licensor as mentioned earlier.'''

In [62]:
generate(query)

**Clause 7:** The licensee cannot make any changes to the building or its layout without the landlord's written permission.

**Clause 8:** The licensee does not have any rights as a tenant and cannot rent out, sell, or mortgage the property. to anyone else. They also cannot borrow money against the property.

**Clause 9:** The landlord can visit the property at reasonable times to check on it.

**Clause 10:** If the licensee breaks any of the rules in the agreement or doesn't pay the rent on time, the landlord can cancel the agreement and ask the licensee to leave. The licensee can also give one month's notice to leave the property.

In [80]:
query = "what are some actions I can take as a landlord?"

In [81]:
generate(query)

- Make an order that the tenant or any such person as aforesaid shall place the landlord in vacant possession of such premises or part thereof
- Proceed to take action under section 45
- Give notice to the landlord of the enquiry to be made in the application under sub-section (3) or during the pendency of such enquiry

In [84]:
query = "what are some powers of the landlord as per this law?"

In [85]:
generate(query)

The provided text does not specify any powers of the landlord.

In [96]:
query = "is it compulsory to register the agreement?"

In [97]:
generate(query)

Yes, it is compulsory to register the agreement.

According to the context, "The responsibility of getting such agreement registered shall be on the landlord and in the absence of the written registered agreement, the contention of the tenant about the terms and conditions subject to which premises have been given to him by the landlord on leave and licence or have been let to him, shall prevail, unless proved otherwise."

In [92]:
query = "what are my rights under this act as an landlord?"

In [95]:
generate(query)

The provided text does not specify the rights of a landlord under this act. mentioned act. Therefore, I cannot answer the question.

In [98]:
query = "for how long can I make the agreement for?"

In [99]:
generate(query)

The provided contexts do not contain any information on the duration for which an agreement can be made. Therefore, I cannot answer this question from the provided context.

In [100]:
augment_prompt(query)

"Using the contexts below, answer the question in detail.\n\n    Contexts:\n    commencement of this Act, shall be in writing and shall be registered under the \nRegistration Act, 1908. \n(2) The responsibility of getting such agreement registered shall be on the landl ord and in \nthe absence of the written registered agreement, the contention of the tenant about  the \nterms and conditions subject to which a premises have been given to him by the landlord \non leave and licence or have been let to him, shall prevail, unless proved otherwise.\nthe landlord rent for one month. If the tenant gives such intimation and makes the \ndeposit, the landlord shall, on completion of the work of repairs, place the tenant in \noccupation of the premises or part thereof on the terms and conditions existing on the \ndate of the passing of the decree for eviction. If the tenant fails to give suc h intimation \nand to make the deposit, the tenant's right to occupy the premises shall terminate .\n(2) I